In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GRU
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
data = []
for i in range(14):
  try:
    data.append(pd.read_csv(f"{i}.csv"))
  except:
    pass

data

[     index    time     acc x  ...  sin ori y  sin ori z  user input
 0        0   0.000  0.205414  ...   0.000000   0.203847           0
 1        1   0.049  0.248886  ...   0.023918   0.255799           0
 2        2   0.099  0.274536  ...   0.028970   0.269050           0
 3        3   0.149  0.318893  ...   0.017733   0.352593           0
 4        4   0.199  0.331879  ...   0.002718   0.382299           0
 ..     ...     ...       ...  ...        ...        ...         ...
 226    226  12.410 -0.010376  ...  -0.996395  -0.010338           0
 227    227  12.460  0.054672  ...  -0.996355   0.056031           0
 228    228  12.509 -0.014191  ...  -0.996315  -0.014106           0
 229    229  12.559  0.017059  ...  -0.995062   0.017362           0
 230    230  12.609 -0.022629  ...  -0.994028  -0.024373           0
 
 [231 rows x 9 columns],
      index    time     acc x  ...  sin ori y  sin ori z  user input
 0        0   0.000  0.262924  ...   0.000000   0.274964           0
 1     

In [16]:
# prompt: make a dataset to train GRU, the input is acc x, acc y, acc z, sin ori x, sin ori y, sin ori z and the output is user input
X = []
y = []

window_size = 10 # Define a window size for your sequences

for df in data:
  # Assuming your dataframe has columns 'acc x', 'acc y', 'acc z', 'sin ori x', 'sin ori y', 'sin ori z', and 'user input'
  # You may need to adjust column names based on your actual data
  acc_x = df['acc x'].values
  acc_y = df['acc y'].values
  acc_z = df['acc z'].values
  sin_ori_x = df['sin ori x'].values
  sin_ori_y = df['sin ori y'].values
  sin_ori_z = df['sin ori z'].values
  user_input = df['user input'].values

  for i in range(len(df) - window_size):
    X.append(np.array([
        acc_x[i:i+window_size],
        acc_y[i:i+window_size],
        acc_z[i:i+window_size],
        sin_ori_x[i:i+window_size],
        sin_ori_y[i:i+window_size],
        sin_ori_z[i:i+window_size]
    ]).T) # Transpose to get shape (window_size, num_features)
    y.append(user_input[i+window_size])

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1180, 10, 6)
y_train shape: (1180,)
X_test shape: (296, 10, 6)
y_test shape: (296,)


# 1. GRU model that has a 6 dimension vector input(acc x, y, z & ori x, y, z)

In [22]:
model = Sequential()
model.add(GRU(8, input_shape=(None, 6)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_8 (GRU)                     │ (None, 8)              │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 393 (1.54 KB)

 Trainable params: 393 (1.54 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test))

Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.5710 - loss: 0.7104 - val_accuracy: 0.8480 - val_loss: 0.5188
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8212 - loss: 0.5096 - val_accuracy: 0.9189 - val_loss: 0.3606
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9276 - loss: 0.3468 - val_accuracy: 0.9595 - val_loss: 0.2421
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9541 - loss: 0.2229 - val_accuracy: 0.9696 - val_loss: 0.1691
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9599 - loss: 0.1734 - val_accuracy: 0.9662 - val_loss: 0.1356
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9635 - loss: 0.1422 - val_accuracy: 0.9662 - val_loss: 0.1233
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9645 - loss: 0.1302 - val_accuracy: 0.9662 - val_loss: 0.1182
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9671 - loss: 0.1182 - val_accuracy: 0.9662 - va

In [25]:
model.evaluate(X_test,y_test)
model.save("gru-6d.h5")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9811 - loss: 0.0740 


# 2. GRU model that has a 3 dimension vector input(acc x, y, z)

In [9]:
# model = Sequential()
# model.add(GRU(8, input_shape=(None, 3)))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                     │ (None, 8)              │           312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)